# Jet Engine Performance Model (Steady State)

by Noah Compton, Isabel Husted, and Jose Roque

## Parts -or- Components
Parts are stored in independent files, and they all define a `class` with the Part name, for example, the file `compressor.py` will have the definition of the `class` Compressor.

For example, to import an specific Compressor, we export the Compressor `class` from the file `compressor.py`

In [21]:
from compressor import Compressor

Cmp20 = Compressor(name='Cmp20',eff_poly = {'value': 0.9, 'units': '-'}, PR = {'value': 20., 'units': '-'})

KeyError: 0

In [20]:

exp = "Cmp20.PR = {'value': 20., 'units': '-' }"
print(exp)
exec(exp)

interested = [
                'name'     , 'inlet'    , 'outlet'   , 
                'Pt_in'    , 'Tt_in'    , 'W_in'     , 
                'Pt_out'   , 'Tt_out'   , 'W_out'    , 
                'PR'       , 'TR'       , 'N_mech'   ,
                'eff_poly' , 'eff_mech' , 'eff_isen' ,
                ]

print(interested)

'name' in interested
# if property == 'name':
#     self.name = values

# elif property in interested:
#     exp1 = "self.Pt_in['value'] = value"
#     exp2 = "self.Pt_in['units'] = units"

#     exec(exp1)

#     if len(values) == 2:
#         exec(exp2)

#     elif len(values) < 2:
#         raise ValueError("Some of the inputs are missing units")

# else:
#     raise Warning("Some inputs were not expected, ignoring extra inputs")

Cmp20.PR = {'value': 20., 'units': '-' }
['name', 'inlet', 'outlet', 'Pt_in', 'Tt_in', 'W_in', 'Pt_out', 'Tt_out', 'W_out', 'PR', 'TR', 'N_mech', 'eff_poly', 'eff_mech', 'eff_isen']


True

## Part List
To start building the engine model, all parts must be loaded from its specified file:

In [ ]:
# Part List
from inlet      import Inlet
from fan        import Fan
from bypass     import Bypass
from compressor import Compressor
from burner     import Burner
from turbine    import Turbine
from mixer      import Mixer
from nozzle     import Nozzle

or simply by creating a separate file named `part_list.py` and list all parts as shown above, we can import as:

In [ ]:
# Part List
from part_list import *

## Processes
Processes are `functions` that will be used throughout the Parts, make them iteract with each other and calculate certain paramenters. 

Similar to part_list, processes are defined in an independent file `processes.py`

In [ ]:
# Processes
from processes import *

For example, the function `check_units` in the processes file, will compare the units of two variables, this way when any class tries to add or substact variables it will check if they are compatible with each other to avoid calculation errors:

In [ ]:
var1 = {'value': 1.    , 'units': 'kg/s'} 
var2 = {'value': 2.    , 'units': 'kg'  } 
var3 = {'value': float , 'units':  str  }

check_units(var1, var2)

var3['value'] = var1['value'] + var2['value']
var3['units'] = var1['units']

Units for `var1` were kg/s and `var2` kg ... you can't add different dimensions!

Now, modifying `var2` to be kg/s ... 

In [ ]:
var1 = {'value': 1.   , 'units': 'kg/s'} 
var2 = {'value': 2.   , 'units': 'kg/s'}
var3 = {'value': float, 'units':  str  }

check_units(var1, var2)

var3['value'] = var1['value'] + var2['value']
var3['units'] = var1['units']

var3

## Engine Definition
After all parts are loaded, then all the components in the engine are defined:

In [ ]:
# Inlet
Int00 =      Inlet(name='Int10')

# Core
Fan10 =        Fan(name='Fan10')
Cmp20 = Compressor(name='Cmp20')
Brn30 =     Burner(name='Brn30')
Trb40 =    Turbine(name='Trb40')

# Bypass
Byp13 =     Bypass(name='Byp13')
Byp15 =     Bypass(name='Byp15')

# Mixing Plane
Mix50 =      Mixer(name='Mix50')

# Nozzle / Exhaust
Noz70 =     Nozzle(name='Noz70')

## Test Data -or- Design Parameters
After each component is defined, then known values are assigned, which can come from test data or design parameters

In [ ]:
# Test Data / Design Parameters
# Inlet, Station: 00
Int00.u_in     = {'value': 600.    , 'units': 'm/s'  }
Int00.W_in     = {'value': 25.     , 'units': 'kg/s' }

# Compressor, Station: 20
Cmp20.PR       = {'value': 20.     , 'units': '-'    }
Cmp20.eff_poly = {'value': 0.9     , 'units': '-'    }

# Burner, Station: 30
Brn30.PR       = {'value': 0.95    , 'units': '-'    }
Brn30.eff_mech = {'value': 0.98    , 'units': '-'    }
Brn30.Tt_out   = {'value': 1673.   , 'units': 'K'    }

# Nozzle, Station: 70
Noz70.u_out    = {'value': 735.    , 'units': 'm/s'  }
Noz70.W_out    = {'value': 25.1070 , 'units': 'kg/s' }


## Performance Parameters:
processes contain a few functions to calculate performance paramenters of the system, such as:
1) Gross Thrust -> `gross_thrust(nozzle)`
2) Ram Drag -> `ram_drag(inlet)`
3) Net Thrust -> `net_thrust(inlet, nozzle)`
4) TSFC (Thrust-specific-fuel-conpsumption) -> `tsfc(inlet, burner, nozzle)`
5) Thermal, Propulsive and Overall System Efficiency -> `efficiency(inlet, burner, nozzle)`

All functions will take as input the specific class

### Net Thrust
Net Thrust can be calculated using net_thrust(inlet: Inlet, nozzle: Nozzle):
- Format `net_thrust(inlet, nozzle)`

The function will take as input the `inlet` and `nozzle` of the Engine and calculate the net thrust for the given condition

In [ ]:
Fn    = net_thrust(Int00, Noz70)
Fn